In [21]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from dotenv import load_dotenv
load_dotenv()
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.core import StorageContext, load_index_from_storage

In [7]:
documents = SimpleDirectoryReader(
     input_files=["./data/hotels_50_entries_countries.csv"], encoding="latin-1"
).load_data()

In [8]:
# define LLM

llm = OpenAI(temperature=0, model="gpt-3.5-turbo-0125")
Settings.llm = llm
Settings.chunk_size = 512

In [9]:
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
)

In [22]:
# index.storage_context.persist(persist_dir="./storage")
index = load_index_from_storage(storage_context)

In [23]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)
response = query_engine.query(
    "What are some hotels in Sarande, Albania",
)

In [24]:
display(Markdown(f"{response}"))

There are several hotels in Sarande, Albania, such as Hotel Butrinti, Hotel Bahamas, Hotel Kaonia, and Hotel Saranda International.

In [25]:
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)
response = query_engine.query(
    "Facilities available in Las Tholas Hotel, Uyuni, Bolivia",
)

In [26]:
display(Markdown(f"{response}"))

The facilities available at Las Tholas Hotel in Uyuni, Bolivia include a 24-hour front desk, laundry facilities, free WiFi throughout the property, a tour desk, and a shared kitchen. Additionally, the hotel offers a terrace, designated smoking areas, and lockers for guests.

In [16]:
# NOTE: can take a while!
# new_index = KnowledgeGraphIndex.from_documents(
#     documents,
#     max_triplets_per_chunk=2,
#     include_embeddings=True,
# )

In [27]:
from pyvis.network import Network

# Get the networkx graph
g = index.get_networkx_graph()

# Create the Pyvis network
net = Network(notebook=True, cdn_resources="in_line", directed=True)

# Load the networkx graph into the Pyvis network
net.from_nx(g)

# Generate HTML content
html_content = net.generate_html()

# Write the HTML content to a file with UTF-8 encoding
with open("example_hotels.html", "w", encoding="utf-8") as f:
    f.write(html_content)

# Display the generated HTML file in the notebook
from IPython.display import IFrame
IFrame("example_hotels.html", width=900, height=600)
